In [1]:
import pandas as pd
import re
import requests
import difflib
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [20]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get('https://fantasyfootballcalculator.com/adp', headers=header)
ffc = pd.read_html(r.text, header = 0); ffc = ffc[0]

ffc = ffc[['Pick', 'Name', 'Overall', 'Std.Dev', 'High', 'Low']]
ffc = ffc.rename(columns = {'Name': 'name'})
ffc['name'].to_csv('ffc_names.csv')


r = requests.get('https://www.fantasypros.com/nfl/adp/overall', headers=header)

fp = pd.read_html(r.text)[0]
fp['Player Team (Bye)'] = fp.iloc[:,1].astype('str')
name = []
for i in range(0,len(fp)):
    try: 
        name.append (re.findall('(.*?)\s[A-Z][A-Z]', fp.iloc[i,1])[0])
    except:
        name.append("")
    

fp['name'] = name
fp = fp.drop('Player Team (Bye)', axis = 1)

stopwords = [" Jr.", " II", " III", " IV", "Defense"]



fp['name'].apply(lambda x: [item for item in x if item not in stopwords])

pat = r'\b(?:{})\b'.format('|'.join(stopwords))
fp['name'] = fp['name'].str.replace(pat, '')
ffc['name'] = ffc['name'].str.replace(pat, '')
fp['name'] = fp['name'].str.replace('.', '')
ffc['name'] = ffc['name'].str.replace('.', '')


fp['name'] = fp['name'].str.upper()
ffc['name'] = ffc['name'].str.upper()

rem_name = []
remainder = ffc[-ffc['name'].isin(fp['name'])]

for i in range(0, len(remainder)):
    rem_name.append( difflib.get_close_matches(remainder.iloc[i,1], fp['name'], cutoff = .55)[0])

indexer = ffc[-ffc['name'].isin(fp['name'])].index
ffc.loc[indexer, "name"] = rem_name



df = ffc.merge(fp)
proj = pd.read_csv('../projections.csv')
rem_name = []
for i in range(0, len(df)):
    rem_name.append( difflib.get_close_matches(df.iloc[i,1], proj['name'], cutoff = .15)[0])

df["name"] = rem_name
df = df[['name', 'Overall', 'AVG', 'Std.Dev', 'High', 'Low']]


df['High'] = df['High'] + (12 * (df['High'] - 1.01) ) -.01
df['Low'] = df['Low'] + (12 * (df['Low'] - 1.01) )

df.to_csv('../adp.csv')


PermissionError: [Errno 13] Permission denied: '../adp.csv'

In [21]:
df.to_csv('../adp.csv')

In [10]:
proj['name']

0                       BUFFALO
1                  INDIANAPOLIS
2                         MIAMI
3                   NEW ENGLAND
4                      NEW YORK
5                    CINCINNATI
6                     CLEVELAND
7                     TENNESSEE
8                  JACKSONVILLE
9                    PITTSBURGH
10                       DENVER
11                  KANSAS CITY
12                      OAKLAND
13                  LOS ANGELES
14                      SEATTLE
15                       DALLAS
16                     NEW YORK
17                 PHILADELPHIA
18                      ARIZONA
19                   WASHINGTON
20                      CHICAGO
21                      DETROIT
22                    GREEN BAY
23                    MINNESOTA
24                    TAMPA BAY
25                      ATLANTA
26                     CAROLINA
27                  LOS ANGELES
28                  NEW ORLEANS
29                SAN FRANCISCO
30                    BALTIMORE
31      

In [329]:
df

,name,Overall,AVG,Std.Dev,High,Low
0,SAQUON BARKLEY,1.5,1.0,0.7,1.00,1.40
1,ALVIN KAMARA,2.4,2.3,0.8,1.00,1.53
2,CHRISTIAN MCCAFFREY,2.5,2.7,0.9,1.00,1.53
3,EZEKIEL ELLIOTT,4.5,4.0,1.4,1.00,2.31
4,DAVID JOHNSON,5.2,6.0,1.3,1.26,2.31
5,DEANDRE HOPKINS,6.0,5.7,1.8,1.00,2.44
6,LE'VEON BELL,7.3,6.7,2.3,1.00,14.53
7,DAVANTE ADAMS,7.9,8.7,2.0,1.00,14.14
8,NICK CHUBB,8.5,14.0,2.3,1.13,14.53
9,JAMES CONNER,9.8,9.3,2.2,1.39,14.53


In [14]:
r = requests.get('https://www.fantasypros.com/nfl/adp/overall', headers=header)


In [15]:
r

<Response [200]>

In [16]:
r.text

'<!DOCTYPE html>\n<html lang="en">\n\n<head>\n    <title>Average Draft Position (ADP) - Fantasy Football 2019 | FantasyPros</title>\n    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\n    <meta name="description" content="FantasyPros combines Average Draft Position rankings from major league commissioner sites to produce a consensus ADP.">\n<link rel="canonical" href="https://www.fantasypros.com/nfl/adp/overall" />\n\n    <meta property="fb:pages" content="184352014941166"/>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\n    <link href=\'https://fonts.googleapis.com/css?family=Open+Sans:200,400,600,700,800|Raleway:200,400,600,700,800\' rel=\'stylesheet\' type=\'text/css\'>\n    <link type="text/css" href="//www.fantasypros.com/assets/css/font-awesome.4.6.3.min_bootstrap.3.partial.min.css" rel="stylesheet">\n\n    <link type="text/css" href="//cdn.fantasypros.com/assets/css/min/style-3380bcb8df.css" rel="stylesheet">\n\n    <style 